# openWithHaloFinder.ipynb

This is an advanced tutorial using FIREreader, be warned!!

This notebook is best used on Stampede2, where the halo file and snapshot directories live. You can run this notebook, and host a Firefly server, on Stampede by following the instructions [here](https://github.com/ageller/Firefly/wiki/Hosting-Firefly-on-a-Cluster-Environment). 

In this notebook, we open the AHF halo files saved on Stampede and offset the snapshot coordinates, as well as convert them to physical units, to put the center of the main halo at our origin. This is optional, since you can always fly within Firefly to a point and set that as your origin, but more convenient (and exact!). 

We also calculate the radius from the halo center for each particle and update the filter keys so we can interactively filter by radius from within Firefly. 

#### Importantly, we do **not** call the `reader.run()` method, which would not give us the flexibility required to change our units/calculate the radii & temperature before we output to JSON. 



In [2]:
%load_ext autoreload
%autoreload 2

from FIREreader import FIREreader
import numpy as np
import os
import h5py

In [3]:
## initialize reader object and choose simulation to run
reader = FIREreader()
reader.directory = "/Users/agurvich/research/snaps/m12i_res7100/output"
reader.snapnum = 600
## could read this from snapshot times
current_redshift=0

## Open the AHF Halo file and extract the halo center and other parameters

In [4]:
def load_AHF(directory,snapnum,current_redshift,hubble = 0.702):
        path = os.path.join(directory,'../halo/ahf/halo_00000_smooth.dat')
        
        ## find column numbers without having to count
        names_to_read = ['snum','Xc','Yc','Zc','Rvir','v_esc','Rstar0.5']
        
        ## load the first line of the datafile
        names = list(np.genfromtxt(path,skip_header=0,max_rows = 1,dtype=str))
        cols = []

        ## find the column each name appears in
        for name in names_to_read:
            cols+=[names.index(name)]

        ## load the rest of the file
        sns,xs,ys,zs, rvirs, vescs, rstar_halfs = np.genfromtxt(
            path,delimiter='\t',usecols=cols,unpack=1,skip_header=1)

        ## which row do I care about? make an index array
        index = sns==snapnum
        if np.sum(index)==0:
            ## snapnum is not in this halo file
            raise IOError
            
        ## presumably in comoving kpc/h 
        halo_center = np.array([xs[index],ys[index],zs[index]])/hubble*(1/(1+current_redshift))
        halo_center = halo_center.reshape(3,)

        ## convert other quantities one might care about from comoving kpc to pkpc
        rvir = rvirs[index][0]/hubble/(1+current_redshift)
        vesc = vescs[index][0]
        rstar_half = rstar_halfs[index][0]/hubble/(1+current_redshift)
        return halo_center, rvir, vesc, rstar_half
    
def getTemperature(U_code,y_helium,ElectronAbundance):
    """U_codes = res['u']
        y_heliums = res['z'][:,1]
        ElectronAbundance=res['ne']"""
    U_cgs = U_code*1e10
    gamma=5/3.
    kB=1.38e-16 #erg /K
    m_proton=1.67e-24 # g
    mu = (1.0 + 4*y_helium) / (1+y_helium+ElectronAbundance) 
    mean_molecular_weight=mu*m_proton
    return mean_molecular_weight * (gamma-1) * U_cgs / kB # kelvin

In [4]:
## open the halo file and find the center
halo_center,rvir,vesc,rstar_half = load_AHF(reader.directory,reader.snapnum,current_redshift)
print halo_center,rvir

[ 41875.75818899  44122.37307211  46257.47577379] 273.803418803


## Setup the reader configuration and load data

In [5]:
## decide which part types to save to JSON
reader.returnParts = ['PartType0', 'PartType4']

## choose the names the particle types will get in the UI
reader.names = {'PartType0':'Gas', 
                  'PartType1':'HRDM', 
                  'PartType2':'LRDM', 
                  'PartType4':'Stars' }

In [6]:
#define the defaults; this must be run first if you want to change the defaults below
reader.defineDefaults()

## by what factor should you sub-sample the data (e.g. array[::decimate])
decimate = [100., 1000.]

In [7]:
## load in the data from hdf5 files and put it into reader.partsDict
for i,p in enumerate(reader.returnParts):
    reader.decimate[p] = decimate[i]
    reader.returnKeys[p] = ['Coordinates', 'Density','Velocities']
    #Note: you should only try to filter on scalar values (like density).  
    #The magnitude of the Velocities are calculated in Firefly, and you will automatically be allowed to filter on it
    reader.addFilter[p] = [False, True, False]
    ## tell it to do the log of density when filtering
    reader.dolog[p] = [False, True, False]
    
    
    #NOTE: all dictionaries in the "options" reference the swapped names (i.e., reader.names) you define above.  
    #If you don't define reader.names, then you can use the default keys from the hdf5 files 
    #(but then you will see those hdf5 names in the Firefly GUI)
    pp = reader.names[p]
    ## set the initial size of the particles when the interface loads
    reader.options['sizeMult'][pp] = 0.3
    
## set the default colors when the interface loads
reader.options['color'] = {'Gas':  [1., 0., 0., 1.],  
                           'HRDM': [1., 1., 0., 0.1],  
                           'LRDM': [1., 1., 0., 0.1],  
                           'Stars':[0., 0., 1., 0.1]} 

## set the camera center to be at the origin (defaults to np.mean(Coordinates) otherwise)
##     later on we subtract out halo_center from coordinates but could instead make this halo_center
reader.options['center'] = np.array([0., 0., 0.])

## initialize filter flags and options
reader.defineFilterKeys()

## load in return keys from snapshot
filenames_opened = reader.populate_dict()

/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
This is a cosmological snapshot... converting to physical units
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.1.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.2.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.3.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
This is a cosmological snapshot... converting to physical units
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.1.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.2.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.3.hdf5
PartType4 has no Density


### Let's calculate the galactocentric radius, offset the coordinates by it while we're at it, then add the array to Firefly using the `addtodict` method

In [8]:
hubble=.702

## while we're at it, let's just shift all the coordinates relative to the main halo center
reader.partsDict['PartType0']['Coordinates']=reader.partsDict['PartType0']['Coordinates']-halo_center ## both already in physical coordinates
reader.partsDict['PartType4']['Coordinates']=reader.partsDict['PartType4']['Coordinates']-halo_center ## both already in physical coordinates

## calculate the radius from the halo center
gas_radii = np.sum(reader.partsDict['PartType0']['Coordinates']**2,axis=1)**0.5
star_radii = np.sum(reader.partsDict['PartType4']['Coordinates']**2,axis=1)**0.5

## add new radius array to the dictionary using addtodict method
reader.addtodict(reader.partsDict,None,'PartType0','Radius',0,0,vals=gas_radii, filterFlag = True)
reader.addtodict(reader.partsDict,None,'PartType4','Radius',0,0,vals=star_radii, filterFlag = True)

### Let's convert the density to physical units

In [9]:
# Code mass -> g , (code length)^-3 -> cm^-3 , g -> nHydrogen
DENSITYFACT=2e43*(3.086e21)**-3/(1.67e-24)
reader.partsDict['PartType0']['log10Density'] = reader.partsDict['PartType0']['log10Density']+np.log10(DENSITYFACT)

### Now let's load necessary supplemental data to calculate the temperature

In [10]:
## add temperature as a filtered quantity within the parts dict, but only for gas
all_gas_temperature = np.array([])
all_star_masses = np.array([])
for fname in reader.loadedHDF5Files:
    print fname
    with h5py.File(fname,'r') as handle:
        ## load necessary arrays to calculate temperature
        gas_group = handle['PartType0']
        InternalEnergy = np.array(gas_group['InternalEnergy'])
        ElectronAbundance = np.array(gas_group['ElectronAbundance'])
        Metallicity = np.array(gas_group['Metallicity'])
        
        ## calculate the temperature and append it to total array
        all_gas_temperature=np.append(
            all_gas_temperature,
            getTemperature(
                InternalEnergy,
                Metallicity[:,1],
                ElectronAbundance)
        )
        
        ## save stellar masses for vcom below
        all_star_masses=np.append(
            all_star_masses,
            np.array(handle['PartType4/Masses'])
        )
    
## track the Temperature array, do the log, and add it to be filtered
reader.addtodict(reader.partsDict,None,'PartType0','Temperature',sendlog = 1, sendmag = 0,vals = all_gas_temperature, filterFlag = True)

/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.1.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.2.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.3.hdf5


### Let's remove halo "CoM" velocity from velocities so that velocity vectors are accurate

In [11]:
## find the nearby stars
near_star_indices = star_radii < rstar_half

## calculate vcom
near_star_vcom = (
    np.sum(all_star_masses[near_star_indices][:,None]
    *reader.partsDict['PartType4']['Velocities'][near_star_indices],axis=0)
    /np.sum(all_star_masses[near_star_indices])
)

print(near_star_vcom,'kms')

## now let's remove it from the particle velocities
reader.partsDict['PartType4']['Velocities']-=near_star_vcom
reader.partsDict['PartType0']['Velocities']-=near_star_vcom


(array([-51.02232016,  74.50281519,  96.03350012]), 'kms')


In [12]:
## finish up, let's shuffle + decimate, add the GUI friendly names, and create our final JSON!
reader.shuffle_dict()
reader.swap_dict_names()
reader.createJSON()

decimating and shuffling ...
decimating and shuffling ...
('dataDir', None)
writing JSON files ...
Gas
Stars
/Users/agurvich/research/repos/Firefly/data/m12i_res7100_600/FIREdataOptions.json


In [1]:
reader.createJSON()

NameError: name 'reader' is not defined